In [1]:
from datamarket import *

In [2]:
dimensions = 4
dimensions_size = 30

In [3]:
# 1st dimension is A, 2nd dimension is B, ...
def get_dimension_name(d):
    return chr(ord('A') + d)

def generate_dimension_table(d):
    name = get_dimension_name(d)
    df = pd.DataFrame()
    df[name] = range(dimensions_size)
    df[name + "_att"] = choices(range(50), k=dimensions_size)
    return df

In [4]:
dimention_tables = []
for i in range(dimensions):
    dimention_tables.append(generate_dimension_table(i))

fact_table = pd.DataFrame()
for d in range(dimensions):
    name = get_dimension_name(d)
    fact_table[name] = [item for item in range(dimensions_size) for i in range(int(pow(dimensions_size,d)))] * int(pow(dimensions_size,dimensions - 1 - d))

# join fact table with dimension tables
join_result = None
for i in range(dimensions):
    if i == 0:
        join_result = pd.merge(fact_table, dimention_tables[0])
    else:
        join_result = pd.merge(join_result, dimention_tables[i])

In [5]:
join_result["result"] = 0
for i in range(dimensions):
    name = get_dimension_name(i)
    join_result["result"] += join_result[name + "_att"] * join_result[name + "_att"]
join_result

,A,B,C,D,A_att,B_att,C_att,D_att,result
0,0,0,0,0,13,27,8,38,2406
1,1,0,0,0,2,27,8,38,2241
2,2,0,0,0,31,27,8,38,3198
3,3,0,0,0,11,27,8,38,2358
4,4,0,0,0,4,27,8,38,2253
...,...,...,...,...,...,...,...,...,...
809995,25,29,29,29,6,49,17,34,3882
809996,26,29,29,29,2,49,17,34,3850
809997,27,29,29,29,36,49,17,34,5142
809998,28,29,29,29,41,49,17,34,5527


In [6]:
# this vary the buyer and seller data cardinality
train = pd.concat([join_result])
seller =  pd.concat([join_result])
# train = pd.concat([join_result,join_result,join_result,join_result])

In [8]:
%%time
# for market bootstrap
h_data = agg_dataset()
h_data.load(seller, ["A_att", "B_att", "C_att", "result"], ["A"], "gender")
h_data.compute_agg()

CPU times: user 305 ms, sys: 208 ms, total: 513 ms
Wall time: 511 ms


In [9]:
train_data = agg_dataset()
train_data.load(train, ["A_att", "B_att", "C_att", "result"], ["A"], "gender")
train_data.compute_agg()

In [10]:
%%time
# for vertical augmentation with factorized learning
horizontal_cov = train_data.covariance + h_data.covariance
parameter = linear_regression(horizontal_cov,["gender:A_att", "gender:B_att"], "gender:result")

CPU times: user 488 µs, sys: 148 µs, total: 636 µs
Wall time: 645 µs


In [12]:
%%time
# this is for naively train linear regression
train = pd.concat([join_result, join_result, join_result, join_result, join_result])
reg = LinearRegression().fit(train[["A_att", "B_att", "C_att", "D_att"]], train["result"])

CPU times: user 959 ms, sys: 1.02 s, total: 1.98 s
Wall time: 788 ms


In [28]:
vertical_aug =  pd.concat([dimention_tables[2]])
vertical_data = agg_dataset()
vertical_data.load(vertical_aug, ["C_att"], ["C"], "gender")
vertical_data.remove_redundant_columns()
vertical_data.compute_agg()

In [29]:
train = pd.concat([join_result])
train_data = agg_dataset()
train_data.load(train, ["A_att", "B_att", "result"], ["C"], "gender")
train_data.compute_agg()

In [30]:
%%time
# vertical augmentation
join_cov = connect(train_data, vertical_data, "C").sum()
linear_regression(join_cov,["gender:A_att", "gender:B_att", "gender:C_att"], "gender:result")

CPU times: user 51.9 ms, sys: 23.9 ms, total: 75.8 ms
Wall time: 74 ms


/home/jupyter/arda/arda-datasets/kitana/datamarket.py:384: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  join.drop('cov:c_y', 1, inplace=True)


array([4.25317685e+01, 5.14802745e+01, 1.23321207e-04, 1.06280850e+03])

In [20]:
# vertical_aug =  pd.concat([dimention_tables[2]])
train = pd.concat([join_result,join_result,join_result,join_result])

In [21]:
%%time
join = pd.merge(train,vertical_aug, left_on="C", right_on="C")
reg = LinearRegression().fit(join[["A_att", "B_att", "C_att_x", "D_att"]], join["result"])

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 106 ms


In [31]:
# vary the domain size of join key
dom = 202500*4
join_result["C"] = [item for item in range(dom) for i in range(2)][:810000]
dom

810000

In [32]:
train_data = agg_dataset()
train_data.load(join_result.copy(), ["A_att", "B_att", "D_att","result"], ["C"], "gender")
train_data.compute_agg()

In [39]:
vertical_aug = pd.DataFrame()
vertical_aug["C"] = [item for item in range(dom)]
vertical_aug["C_att"] = choices(range(50), k=len(vertical_aug))

vertical_data = agg_dataset()
vertical_data.load(vertical_aug.copy(), ["C_att"], ["C"], "gender")
vertical_data.remove_redundant_columns()
vertical_data.compute_agg()

In [40]:
%%time
join_cov = connect(train_data, vertical_data, "C").sum()
linear_regression(join_cov,["gender:A_att", "gender:B_att", "gender:C_att"], "gender:result")

CPU times: user 102 ms, sys: 24.8 ms, total: 127 ms
Wall time: 125 ms


array([ 4.25317847e+01,  5.14802851e+01, -2.10390350e-02,  1.06332605e+03])

In [44]:
%%time
join = pd.merge(join_result,vertical_aug, left_on="C", right_on="C")
reg = LinearRegression().fit(join[["A_att", "B_att", "C_att_x", "D_att"]], join["result"])

CPU times: user 254 ms, sys: 50.3 ms, total: 304 ms
Wall time: 302 ms
